# Projeto Final

Para esse projeto foi considerado a base de dados em Português do Brasil de reviews disponíveis no IMDB.

Esse site contém uma ampla variedade de catálogos de filmes em que as pessoas cadastradas podem realizar comentários com suas opiniões.

Essa base de dados está disponível em: 
* https://huggingface.co/datasets/celsowm/imdb-reviews-pt-br

## Fazendo Importações Necessárias

Para execução do código presente nesse Notebook, será necessário a realização dos imports a seguir. 

Essa linha deve sempre ser executada primeiro.

In [1]:
import os

os.environ['TOKENIZERS_PARALLELISM'] = "false"

import snorkel
import pandas as pd
from snorkel.labeling import (LFAnalysis, PandasLFApplier,filter_unlabeled_dataframe, labeling_function)
from snorkel.labeling.model.label_model import LabelModel
from snorkel.utils import probs_to_preds

snorkel.__version__

'0.10.0'

### Carregando o dataset

In [3]:
df = pd.read_parquet('datasets/train-00000-of-00001-34e817339f28df45.parquet')
df.head()

,id,texto,sentimento
0,1,"Mais uma vez, o Sr. Costner arrumou um filme p...",0
1,2,Este é um exemplo do motivo pelo qual a maiori...,0
2,3,"Primeiro de tudo eu odeio esses raps imbecis, ...",0
3,4,Nem mesmo os Beatles puderam escrever músicas ...,0
4,5,Filmes de fotos de latão não é uma palavra apr...,0


In [4]:
# Indicando o random_state apenas para conseguir refazer com os mesmos resultados.
for row in df.sample(50, random_state=1).itertuples():
    print(f"{row.texto} - {row.id}")
    print('\n')

Assassinatos estão ocorrendo em uma cidade do deserto do Texas. Quem é responsável? Ligeiras novidades de mistério e tensões raciais este último realmente não se encaixa, mas de outra forma estritamente para os fãs de slasher, que irão apreciar a nesga e nudez, que são dois elementos convencionais para estes filmes.Dana Kimmell de Sexta-feira 13ª infâmia estrelas como o malcriado adolescente quase-detetive.1 / 2 da MPAA: Pontuação: R por violência e nesga, nudez e alguma linguagem. - 31656


Prós: NothingCons: EverythingPlot resumo: Uma repórter fêmea corre para um caroneiro que conta suas histórias sobre as mortes de pessoas que foram mortas por zumbis. Revisão: Nunca na minha vida eu me deparei com um filme tão ruim quanto The Zombie Chronicles. Filmado em um orçamento do que parece ser cerca de 20 dólares, o TZC é um filme de terror completamente horrível que conta com atores esquecidos e esquecidos que não poderiam agir para salvar suas vidas, o que é mais grosseiro do que assustad

## Transformando o texto para letras minúsculas

Essa transformação facilitará os posteriores processamentos que serão realizados.

In [5]:
df["texto"] = df["texto"].str.lower()
df

,id,texto,sentimento
0,1,"mais uma vez, o sr. costner arrumou um filme p...",0
1,2,este é um exemplo do motivo pelo qual a maiori...,0
2,3,"primeiro de tudo eu odeio esses raps imbecis, ...",0
3,4,nem mesmo os beatles puderam escrever músicas ...,0
4,5,filmes de fotos de latão não é uma palavra apr...,0
...,...,...,...
49454,49456,"como a média de votos era muito baixa, e o fat...",1
49455,49457,o enredo teve algumas reviravoltas infelizes e...,1
49456,49458,estou espantado com a forma como este filme e ...,1
49457,49459,a christmas together realmente veio antes do m...,1


Também precisaremos remover a coluna sentimento. Nos processamentos que iremos fazer, não precisamos dessa coluna. Assim como vai ser interessante embaralharmos as linhas.

In [16]:
df_reviews = df.drop("sentimento", axis = 1)
df_reviews 

df_copy_reviews = df_reviews
df_copy_reviews

,id,texto
0,1,"mais uma vez, o sr. costner arrumou um filme p..."
1,2,este é um exemplo do motivo pelo qual a maiori...
2,3,"primeiro de tudo eu odeio esses raps imbecis, ..."
3,4,nem mesmo os beatles puderam escrever músicas ...
4,5,filmes de fotos de latão não é uma palavra apr...
...,...,...
49454,49456,"como a média de votos era muito baixa, e o fat..."
49455,49457,o enredo teve algumas reviravoltas infelizes e...
49456,49458,estou espantado com a forma como este filme e ...
49457,49459,a christmas together realmente veio antes do m...


In [7]:
df

,id,texto,sentimento
0,1,"mais uma vez, o sr. costner arrumou um filme p...",0
1,2,este é um exemplo do motivo pelo qual a maiori...,0
2,3,"primeiro de tudo eu odeio esses raps imbecis, ...",0
3,4,nem mesmo os beatles puderam escrever músicas ...,0
4,5,filmes de fotos de latão não é uma palavra apr...,0
...,...,...,...
49454,49456,"como a média de votos era muito baixa, e o fat...",1
49455,49457,o enredo teve algumas reviravoltas infelizes e...,1
49456,49458,estou espantado com a forma como este filme e ...,1
49457,49459,a christmas together realmente veio antes do m...,1


O próximo trecho de código, será um conjunto de funções que servirá para remover e tratar ainda mais as informações do texto, removendo aquilo que não é necessário.

Podemos citar mais de um espaço em branco, acentuações e pontuações.

In [11]:
import re
import unicodedata

def remove_espacos_excessivos(text: str) -> str:
    # \s+ matches one or more whitespace characters
    # ' ' replaces the matched string with a single space
    # strip() removes leading and trailing spaces
    return re.sub(r'\s+', ' ', text).strip() 

def remove_caracteres_repetidos_nao_palavras(text: str) -> str:
    # \W matches any non-word character (equivalent to [^a-zA-Z0-9_ ])
    # \1+ matches one or more occurrences of the same character
    # r'\1' replaces the matched string with a single occurrence of the character
    # strip() removes leading and trailing spaces
    return re.sub(r'(\W)\1+', r'\1', text).strip()

def remove_letras_repetidas(text: str, n_repeat: int = 4) -> str:
    # ([a-z]) matches a lowercase letter
    # \1{'+str(n_repeat)+',}' matches n_repeat or more occurrences of the same letter
    # r'\1' replaces the matched string with a single occurrence of the letter
    # strip() removes leading and trailing spaces
    return re.sub(r'([a-z])\1{'+str(n_repeat)+',}', r'\1', text).strip()

def remove_acentos(text: str) -> str:
    # normalize() converts the text to the NFKD form
    # encode() converts the text to bytes, ignoring non-ASCII characters
    # decode() converts the bytes back to a string, ignoring errors
    return unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')

In [13]:
from sklearn.pipeline import Pipeline 
from sklearn.preprocessing import FunctionTransformer 


pipeline_limpeza_texto = Pipeline([
    ('remove_acentos', FunctionTransformer(remove_acentos)),
    ('remove_espacos_excessivos', FunctionTransformer(remove_espacos_excessivos)),
    ('remove_letras_repetidas', FunctionTransformer(remove_letras_repetidas)),
    ('remove_caracteres_repetidos_nao_palavras', FunctionTransformer(remove_caracteres_repetidos_nao_palavras)),
])

In [17]:
print(df_reviews.shape)
df_reviews.dropna(subset=['texto'], inplace=True)
print(df_reviews.shape)

(49459, 2)
(49459, 2)


In [18]:
df_reviews['texto'] = df_reviews['texto'].apply(pipeline_limpeza_texto.transform)

In [20]:
df_reviews.drop_duplicates(subset=['texto'], inplace=True)
print(df_reviews.shape)

(49045, 2)


In [21]:
df_reviews.drop_duplicates(subset=['id'], inplace=True)
print(df_reviews.shape)

(49045, 2)


In [23]:
df_reviews['texto'].str.len().describe([0.05, 0.1, 0.25, 0.5, 0.75, 0.8, 0.9, 0.95, 0.98, 0.99, 0.999])

count    49045.000000
mean      1281.264533
std        887.186845
min         30.000000
5%         354.000000
10%        503.000000
25%        713.000000
50%        981.000000
75%       1580.000000
80%       1803.000000
90%       2505.000000
95%       3210.800000
98%       4036.240000
99%       4485.680000
99.9%     5126.956000
max       5333.000000
Name: texto, dtype: float64

In [24]:
# keep only reviews between the 10th and 99th percentile
quantils = df_reviews['texto'].str.len().quantile([0.1, 0.99]).values
df_reviews = df_reviews[df_reviews['texto'].str.len() > quantils[0]]
df_reviews = df_reviews[df_reviews['texto'].str.len() <= quantils[1]]
print(quantils)
print(df_reviews.shape)

[ 503.   4485.68]
(43642, 2)


In [26]:
df_reviews.shape

(43642, 2)

# Divisão da base de dados

Precisamos dividir o conjunto de dados que possui mais de 43 mil linhas em 3 conjuntos:
- Conjunto de treinamento
- Conjunto de desenvolvimento
- Conjunto de testes

Para isso, é importante considerar um tamanho bom e de dados aleatórios, com reviews positivas e negativas.

In [28]:
len(df_reviews)

43642

O nosso dataset usado de reviews do IMDb, possui 43642 linhas de reviews. Assim, vamos usar para esse momento, uma amostra com 95% de grau de confiança e 5% de margem de erro.

In [29]:
import math
n = 43642 # Total de reviews
z = 1.96 # Valor tabelado de Z para 95% confidence level
p = 0.5 # Expected proportion of positive reviews. We assume 50%, which is the worst case scenario/
e = 0.05 # Margin of error

tamanho_amostra = math.ceil((z**2 * p * (1-p)) / e**2)
print(f"Tamanho da amostra: {tamanho_amostra}")

Tamanho da amostra: 385


Após embaralharmos o dataset, precisamos de 2 * 385 samples, metade para o conjunto de teste e outra metade para o conjunto de desenvolvimento. Por facilidade, vamos considerar que cada amostra deve conter 400.

In [30]:
df_reviews = df_reviews.sample(frac=1.0, random_state=314)

amostra_desejada = 400

df_dev_test = df_reviews[:amostra_desejada * 2]
df_train = df_reviews[amostra_desejada * 2:]

print('Original dataset size: ', len(df_reviews))
print(f"Train set size: {len(df_train)}")
print(f"Dev/Test set size: {len(df_dev_test)}")


Original dataset size:  43642
Train set size: 42842
Dev/Test set size: 800


In [31]:
df_train.to_parquet('datasets/train.parquet', index=False)
df_dev_test.to_parquet('datasets/dev_test.parquet', index=False)

In [33]:
print(df_train.head())
print(df_dev_test.head())

          id                                              texto
26729  26731  este remake de tv de 1973 do classico de billy...
12976  12978  eu vi esse filme no galway film fleadh no ano ...
14667  14669  este filme e baseado na historia real da dona ...
48435  48437  eu me lembrei disso como sendo um dos meus liv...
42277  42279  outra entrada na categoria "horror de ferias" ...
          id                                              texto
3059    3061  o impensavel aconteceu. tendo assistido pela p...
36357  36359  ei pessoal . realmente nao ha muito a dizer so...
35301  35303  eu achei que isso fosse uma total perda de tem...
1204    1205  e dificil escrever 10 linhas de copia sobre es...
3314    3316  fiquei tao ofendido com esse filme que tive qu...


A partir daqui, como geramos os dados de treinamento, desenvolvimento e testes, não precisaremos mais usar as células anteriores. Podemos apenas usar a partir daqui.

Precisamos apenas executar a célula inicial de importação e remover o caracter underline ("_") usado para não haver conflito com o processo feito anteriormente, caso esteja executando a primeira vez.

In [7]:
df_ = pd.read_parquet('datasets/train-00000-of-00001-34e817339f28df45.parquet')
# df.head()
df_train_ = pd.read_parquet('datasets/train.parquet')
df_dev_test_ = pd.read_parquet('datasets/dev_test.parquet')
print(df_train_.head())
print(df_dev_test_.head())

      id                                              texto
0  26731  este remake de tv de 1973 do classico de billy...
1  12978  eu vi esse filme no galway film fleadh no ano ...
2  14669  este filme e baseado na historia real da dona ...
3  48437  eu me lembrei disso como sendo um dos meus liv...
4  42279  outra entrada na categoria "horror de ferias" ...
      id                                              texto
0   3061  o impensavel aconteceu. tendo assistido pela p...
1  36359  ei pessoal . realmente nao ha muito a dizer so...
2  35303  eu achei que isso fosse uma total perda de tem...
3   1205  e dificil escrever 10 linhas de copia sobre es...
4   3316  fiquei tao ofendido com esse filme que tive qu...


# Recuperando os valores dos sentimentos anotados para os ids específicos no dataset original.

Como temos na nossa base original valores de sentimentos, precisaremos de um valor específico de valores para usar no nosso conjunto de dados de validação e teste.

Assim, precisaremos fazer uma rotina para recuperar os dados do dataframe original e posteriormente colocá-los no nosso conjunto.

Para facilitar as operações, vamos primeiramente construir um dataframe que possui apenas o id e o rótulo específico da linha.

In [8]:
df_rotulado = df.drop("texto", axis = 1)
df_rotulado.head(10)

,id,sentimento
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0
5,6,0
6,7,0
7,8,0
8,9,0
9,10,0


A seguir, faremos um merge apenas da intersecção existente entre o conjunto de dados de teste e validação com a nossa base original rotulada.

In [9]:
df_dev_test_rotulado = df_dev_test.merge(df_rotulado, on = "id", how = "inner")
df_dev_test_rotulado.head(10)

,id,texto,sentimento
0,3061,o impensavel aconteceu. tendo assistido pela p...,0
1,36359,ei pessoal . realmente nao ha muito a dizer so...,0
2,35303,eu achei que isso fosse uma total perda de tem...,0
3,1205,e dificil escrever 10 linhas de copia sobre es...,0
4,3316,fiquei tao ofendido com esse filme que tive qu...,0
5,22608,"""the phenix city story"" e um docudrama brutal ...",1
6,39178,a historia gira em torno de antonio scarface m...,1
7,19793,na linha de fogo nos da um grande jogo de gato...,1
8,23202,"eu nunca li um romance de jacqueline susann, m...",1
9,16309,os primeiros minutos deste filme nao fazem jus...,1


Para evitar perdas, vamos salvar essa base de testes e validação rotulada.

In [10]:
df_dev_test_rotulado.to_parquet("datasets/dev_test_rotulado.parquet")

In [12]:
df_dev_test_rotulado.shape

(800, 3)

Agora, vamos separar o nosso conjunto de dados rotulados, nos conjuntos de desenvolvimento e testes propriamente dito.

In [13]:
from sklearn.model_selection import train_test_split

df_test, df_dev = train_test_split(df_dev_test_rotulado, test_size=0.5, random_state=314, stratify=df_dev_test_rotulado.sentimento)

In [14]:
print(df_test.shape)
print(df_dev.shape)

(400, 3)
(400, 3)


Para ver agora a distribuição de classes entre cada um.

In [16]:
print(df_test.sentimento.value_counts(normalize=True))

sentimento
0    0.505
1    0.495
Name: proportion, dtype: float64


In [17]:
print(df_dev.sentimento.value_counts(normalize=True))

sentimento
0    0.5075
1    0.4925
Name: proportion, dtype: float64


In [18]:
print(df_train.shape, df_dev.shape, df_test.shape)

df_train.drop_duplicates(subset=['id', 'texto'], inplace=True)
df_dev.drop_duplicates(subset=['id', 'texto'], inplace=True)
df_test.drop_duplicates(subset=['id', 'texto'], inplace=True)

print(df_train.shape, df_dev.shape, df_test.shape)

df_train = df_train.query('id not in @df_dev.id or id not in @df_test.id')
df_dev = df_dev.query('id not in @df_train.id or id not in @df_test.id')
df_test = df_test.query('id not in @df_train.id or id not in @df_dev.id')

print(df_train.shape, df_dev.shape, df_test.shape)

(42842, 2) (400, 3) (400, 3)
(42842, 2) (400, 3) (400, 3)
(42842, 2) (400, 3) (400, 3)


In [19]:
df_train.reset_index(drop=True, inplace=True)
df_dev.reset_index(drop=True, inplace=True)
df_test.reset_index(drop=True, inplace=True)

In [22]:
# Save all files for later use
df_test.to_parquet('datasets/processados/test.parquet', index=False)
df_dev.to_parquet('datasets/processados/dev.parquet', index=False)
df_train.to_parquet('datasets/processados/train.parquet', index=False)

df_train.shape, df_test.shape, df_dev.shape

((42842, 2), (400, 3), (400, 3))

# Pipeline de Supervisão Fraca

Agora, temos os nossos conjuntos de testes, desenvolvimento ou validação e treinamento. Podemos então passar para a nossa pipeline de supervisão fraca em que o objetivo será rotular a nossa base de treinamento.

**Obs: Para casos em que estamos continuando daqui, podemos apenas executar a próxima linha que realiza a importação dos conjuntos de dados de testes, desenvolvimento e treinamento.**

In [2]:
df_treinamento = pd.read_parquet("datasets/processados/train.parquet")
df_dev = pd.read_parquet("datasets/processados/dev.parquet")
df_test = pd.read_parquet("datasets/processados/test.parquet")

In [3]:
print(df_treinamento.shape, df_dev.shape, df_test.shape)

(42842, 2) (400, 3) (400, 3)


In [4]:
df_treinamento.head()

,id,texto
0,26731,este remake de tv de 1973 do classico de billy...
1,12978,eu vi esse filme no galway film fleadh no ano ...
2,14669,este filme e baseado na historia real da dona ...
3,48437,eu me lembrei disso como sendo um dos meus liv...
4,42279,"outra entrada na categoria ""horror de ferias"" ..."


In [5]:
df_dev.head()

,id,texto,sentimento
0,41792,"""the kennel murder case"" comeca em uma corrida...",1
1,47040,victor nunez impregna esse relato nao sentimen...,1
2,17237,"um filme incrivel, eu acabei de ver e ja quero...",1
3,20163,angustiante serie sobre a vida em oz - uma pri...,1
4,36150,este deve ser um dos piores filmes suecos ja f...,0


## Passo 1 - Escrever funções de rotulagem (ou labeling functions - LFs)

// TO DO
... DESCREVER O CONCEITO E O QUE SÃO FUNÇÕES DE ROTULAGEM ...

### 1.1 - Explorando o dataset

É importante realizar esse passo para conhecer melhor os nossos dados. As funções de rotulagem são específicas de cada problema. Assim, ao saber minimamente informações sobre os dados, será possível escrever funções que poderão ter um melhor desempenho na tarefa.

In [6]:
for linha in df_treinamento.sample(45, random_state=314).itertuples():
    print(linha.texto + "\n")

quando eu soube que eles estavam fazendo isso, eu fiquei feliz, considerando que o primeiro filme foi muito bom, embora um pouco curto. mas entao me lembrei de alguns dos desastres da sequela da disney que eu ja assisti olhando para voce. pequena sereia 2.anyway eu assisti e, infelizmente, fiquei muito decepcionado. a melhor coisa e que a animacao e excelente. realmente tem aquele polimento especial que os filmes "apropriados" da disney tem. alem disso, o resto e decepcionante. o enredo e serio em todo o lugar. um momento e sobre algo, entao muda completamente para outro enredo e depois muda para outro enredo completamente diferente. isso me lembrou de como o filme family guy era como tres episodios separados, transformados em um filme. eu ri talvez uma vez no maximo. kronk foi muito engracado no filme original, mas nisso ele nao e nada engracado. fique longe deste filme, a menos que alguem lhe empreste gratuitamente.4 / 10

ride with the devil dirigido por ang leecrouching o tiger e o

Alguns indicativos interessantes que podem indicar algum sentimento, "positivo" ou "negativo":
- Existência de palavras positivas "bom" seguido de "mas" pode indicar um sentimento negativo.
- Existência de termos como "infelizmente", "decepciona" ou "decepcionado" pode indicar um sentimento negativo.
- Expressões como "vale a pena" e "must see" (expressão em inglês para recomendações positivas), indicam um sentimento positivo.
- Expressão "deve ver", "maravilha" e "hilário" indicam um sentimento positivo.
- "horrivel", "terrivel", "frustrante", "decepcionante" e "lamentável" indicam um sentimento negativo.
- "Mais engracados", "maiores sitcom", "excelente", "diversão" = positivo
- "obsoleto" = negativo
- "sem inspiracao" = negativo

### 1.2 - Desenvolvendo as LFs

Agora podemos começar a escrever nossas funções de rotulagem.

// TO DO

... INFORMAR O USO DO SNORKEL ...

In [7]:
# Possíveis opiniões para as funções de rotulagem
POSITIVO = 1
NEGATIVO = 0
ABSTENCAO = -1

In [8]:
import re 

regex_a = re.compile(r'(?<!\bnão\s)\bgostei|gostou|gostar|bom|boa\b', re.IGNORECASE)
regex_b = re.compile(r'nao gost', re.IGNORECASE)
regex_c = re.compile(r'(?:mais|maiores) serie|sitcom', re.IGNORECASE)
regex_d = re.compile(r'vale a pena', re.IGNORECASE)
regex_e = re.compile(r'obsolet', re.IGNORECASE)
regex_f = re.compile(r'(?<!\bnão\s)\brecomend\b', re.IGNORECASE)
regex_g = re.compile(r'nao recomend', re.IGNORECASE)
regex_h = re.compile(r'nunca mais', re.IGNORECASE)
regex_i = re.compile(r'(?:sem|nao (:?tem|e|est)) inspi', re.IGNORECASE)
regex_j = re.compile(r'ruim|pessim[oa]|terrivel|horrivel|frustrante|decepc|lamentavel', re.IGNORECASE)
regex_k = re.compile(r'\bbo[ma]\b', re.IGNORECASE)
regex_l = re.compile(r'infeliz|horri|terri', re.IGNORECASE)

In [10]:
from snorkel.labeling import labeling_function

@labeling_function()
def lf_regex_a(x):
    if regex_a.search(x.texto):
        return POSITIVO
    else:
        return NEGATIVO

@labeling_function()
def lf_regex_b(x):
    return POSITIVO if regex_b.search(x.texto) else ABSTENCAO

@labeling_function()
def lf_regex_c(x):
    return POSITIVO if regex_c.search(x.texto) else ABSTENCAO

@labeling_function()
def lf_regex_d(x):
    return POSITIVO if regex_d.search(x.texto) else ABSTENCAO

@labeling_function()
def lf_regex_e(x):
    return NEGATIVO if regex_e.search(x.texto) else ABSTENCAO

@labeling_function()
def lf_regex_f(x):
    if regex_f.search(x.texto):
        return POSITIVO
    else:
        return NEGATIVO

@labeling_function()
def lf_regex_g(x):
    return NEGATIVO if regex_g.search(x.texto) else ABSTENCAO

@labeling_function()
def lf_regex_h(x):
    return NEGATIVO if regex_h.search(x.texto) else ABSTENCAO

@labeling_function()
def lf_regex_i(x):
    return NEGATIVO if regex_i.search(x.texto) else ABSTENCAO

@labeling_function()
def lf_regex_j(x):
    return NEGATIVO if regex_j.search(x.texto) else ABSTENCAO

@labeling_function()
def lf_regex_k(x):
    return POSITIVO if regex_k.search(x.texto) else ABSTENCAO

@labeling_function()
def lf_regex_l(x):
    return NEGATIVO if regex_l.search(x.texto) else ABSTENCAO

Agora, será aplicado as LFs criadas anteriormentes na nossa base.

In [11]:
from snorkel.labeling import PandasLFApplier

lfs = [lf_regex_a, lf_regex_b, lf_regex_c, lf_regex_d, lf_regex_e, lf_regex_f, lf_regex_g, lf_regex_h, lf_regex_i, lf_regex_j, lf_regex_k, lf_regex_l]

applier = PandasLFApplier(lfs=lfs)
L_treinamento = applier.apply(df=df_treinamento)

100%|██████████| 42842/42842 [00:25<00:00, 1682.83it/s]


In [12]:
L_treinamento

array([[ 1, -1, -1, ..., -1,  1, -1],
       [ 1, -1, -1, ..., -1, -1, -1],
       [ 0, -1, -1, ..., -1, -1, -1],
       ...,
       [ 0, -1, -1, ..., -1, -1, -1],
       [ 1, -1, -1, ..., -1,  1, -1],
       [ 0, -1, -1, ..., -1, -1, -1]])

In [13]:
print(L_treinamento.shape)

(42842, 12)


### 1.3 - Avaliando a performance

Vamos avaliar a cobertura das nossas LFs

In [15]:
coverage =  (L_treinamento != ABSTENCAO).mean(axis=0)
print(f'Cobertura das LFs: {coverage}')

Cobertura das LFs: [1.         0.04392885 0.00534522 0.05571635 0.0026376  1.
 0.00985015 0.00826292 0.0052752  0.31473787 0.36023061 0.24968489]


Vamos usar a LFAnalysis que é um conjunto de funções presentes no Snorkel que fornece informações interessantes.

In [16]:
from snorkel.labeling import LFAnalysis

LFAnalysis(L=L_treinamento, lfs=lfs).lf_summary()

,j,Polarity,Coverage,Overlaps,Conflicts
lf_regex_a,0,"[0, 1]",1.000000,1.000000,0.501377
lf_regex_b,1,[1],0.043929,0.043929,0.043929
lf_regex_c,2,[1],0.005345,0.005345,0.005345
lf_regex_d,3,[1],0.055716,0.055716,0.055716
lf_regex_e,4,[0],0.002638,0.002638,0.001237
lf_regex_f,5,[0],1.000000,1.000000,0.501377
lf_regex_g,6,[0],0.009850,0.009850,0.005579
lf_regex_h,7,[0],0.008263,0.008263,0.004178
lf_regex_i,8,[0],0.005275,0.005275,0.002848
lf_regex_j,9,[0],0.314738,0.314738,0.169927


In [18]:
print(LFAnalysis(L=L_treinamento, lfs=lfs).label_coverage())

1.0


Conseguimos cobrir 100%, mas nossas funções de rotulagem tiveram bastante conflito.

### 1.4 - Comparando com o conjunto de desenvolvimento

Agora vamos comparar os resultados que tivemos com o nosso conjunto de desenvolvimento.

In [19]:
L_dev = applier.apply(df=df_dev)

100%|██████████| 400/400 [00:00<00:00, 1629.15it/s]


In [20]:
print(L_dev.shape)

(400, 12)


In [21]:
LFAnalysis(L=L_dev, lfs=lfs).lf_summary(Y=df_dev.sentimento.values)

,j,Polarity,Coverage,Overlaps,Conflicts,Correct,Incorrect,Emp. Acc.
lf_regex_a,0,"[0, 1]",1.0000,1.0000,0.4950,203,197,0.507500
lf_regex_b,1,[1],0.0300,0.0300,0.0300,6,6,0.500000
lf_regex_c,2,[1],0.0100,0.0100,0.0100,2,2,0.500000
lf_regex_d,3,[1],0.0375,0.0375,0.0375,6,9,0.400000
lf_regex_e,4,[0],0.0050,0.0050,0.0025,2,0,1.000000
lf_regex_f,5,[0],1.0000,1.0000,0.4950,203,197,0.507500
lf_regex_g,6,[0],0.0075,0.0075,0.0075,3,0,1.000000
lf_regex_h,7,[0],0.0100,0.0100,0.0050,2,2,0.500000
lf_regex_i,8,[0],0.0025,0.0025,0.0025,1,0,1.000000
lf_regex_j,9,[0],0.3125,0.3125,0.1750,104,21,0.832000


### 1.5 - Algumas visualizações

// TO DO


In [ ]:
normalized_conflict_matrix = LFAnalysis(L=L_train, lfs=lfs).lf_conflicts(normalize_by_overlaps=True)
normalized_conflict_matrix = pd.DataFrame([normalized_conflict_matrix], columns=[lf.name for lf in lfs]).T
normalized_conflict_matrix.columns = ['normalized_conflict']
normalized_conflict_matrix.sort_values(by='normalized_conflict', ascending=False)

conflict_matrix = LFAnalysis(L=L_train, lfs=lfs).lf_conflicts()
conflict_matrix = pd.DataFrame([conflict_matrix], columns=[lf.name for lf in lfs]).T
conflict_matrix.columns = ['conflict']
conflict_matrix.sort_values(by='conflict', ascending=False)

final_cm = pd.concat([conflict_matrix, normalized_conflict_matrix], axis=1)
final_cm

### 1.6 - Usando uma LF que usa um modelo já treinado

A ideia é usar TransferLearning.